In [ ]:
from typing import  Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

from langchain.retrievers import WikipediaRetriever
import requests
from bs4 import BeautifulSoup

llm = ChatOpenAI(temperature=0.1)

def save_to_file(content: str, title: str):
    with open(f"./{title}.txt", "w", encoding="utf-8") as f:
        f.write(content)

def search_wikipedia(keyword: str) -> str:
    retriever = WikipediaRetriever(top_k_results=3, lang="en")
    data_list = retriever.get_relevant_documents(keyword)
    results = "Wikipedia Result\n\n"
    for page_content in data_list:
        results += f"{page_content.page_content}\n\n"
    return results

def scrape_website(url: str) -> str:
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        page_text = "\n".join([p.get_text() for p in paragraphs])
        return page_text
    return "Failed to retrieve content."


class WikipediaToolArgsSchema(BaseModel):
    keyword: str = Field(description="Search Wikipedia using a keyword.")

class WikipediaTool(BaseTool):
    name = "Wikipedia"
    description = "A tool to search for information on Wikipedia using a keyword."
    args_schema: Type[WikipediaToolArgsSchema] = WikipediaToolArgsSchema

    def _run(self, keyword: str):
        wiki_results = search_wikipedia(keyword)
        save_to_file(wiki_results, "wikipedia_result")
        return wiki_results


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaTool(),
    ],
)


prompt = "Research about the XZ backdoor"
agent.invoke(prompt)